In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

movie=pd.read_csv('dataset/tmdb_5000_movies.csv')
credits=pd.read_csv('dataset/tmdb_5000_credits.csv')

In [ ]:
movie.head
movie.shape



In [ ]:
movie=movie.merge(credits,on='title')
movie.shape


In [ ]:
movie.columns


In [ ]:
movie=movie[['movie_id','title','overview','genres', 'keywords','cast', 'crew','original_language',]]


In [ ]:
movie.isnull().sum()


In [ ]:
movie.dropna(inplace=True)
movie.shape

In [ ]:
import ast
def convert(text):
    l=[]
    for i in ast.literal_eval (text): #converting string into list
        l.append(i['name'])
    return l

def convert_c(text):
    l=[]
    c=0
    for i in ast.literal_eval (text):#converting string into list
        if c<5:
         l.append(i['name'])
        c+=1
    
    return l

def convert_cr(text):
    l=[]
  
    for i in ast.literal_eval (text):#converting string into list
        if i['job']=='Director':
          l.append(i['name'])
          break
    
    return l



    

In [ ]:
movie['genres']=movie['genres'].apply(convert)
movie['keywords']=movie['keywords'].apply(convert)
movie['cast']=movie['cast'].apply(convert_c)
movie['crew']=movie['crew'].apply(convert_cr)

In [ ]:
movie.head(2)

In [ ]:
movie['overview']=movie['overview'].apply(lambda x:x.split())
movie.head(2)

In [ ]:
#remove space from name
def remove_space(word):
    l=[]
    for i in word:
        l.append(i.replace(" ",""))

    return l

In [ ]:
movie['genres']=movie['genres'].apply(remove_space)
movie['keywords']=movie['keywords'].apply(remove_space)
movie['cast']=movie['cast'].apply(remove_space)
movie['crew']=movie['crew'].apply(remove_space)


In [ ]:
movie['tags'] = movie['overview'] + movie['genres'] + movie['keywords'] + movie['cast'] + movie['crew']

In [ ]:
movie.head(2)

In [ ]:
movies = movie[['movie_id','title','tags']]

In [ ]:
movies["tags"] = movies["tags"].apply(lambda x: " ".join(map(str, x)))

# Display first few rows
movies.head()

In [ ]:
movies['tags'] = movies['tags'].apply(lambda x:x.lower())

In [ ]:
movies.head(2)

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [ ]:
def steam(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))

    return " ".join(l)
        

In [ ]:
movies['tags'] = movies['tags'].apply(steam)

In [ ]:
movies.iloc[0]['tags']

In [ ]:
#vectorization after that cosine similarity to find nearest vector as suggested
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector=cv.fit_transform(movies['tags']).toarray()

In [ ]:
vector.shape


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sim=cosine_similarity(vector)
sim.shape

In [ ]:
movies[movies['title'] == 'Inception'].index[0]

In [ ]:
def recomand(movie):
    index_list = movies[movies['title'] == movie].index  # Use the variable, not a string
    if index_list.empty:  # Check if the movie exists
        print(f"Movie '{movie}' not found in the dataset.")
        return
    
    index = index_list[0]  # Get the first matching index
    distances = sorted(list(enumerate(sim[index])), reverse=True, key=lambda x: x[1])
    
    print("Recommended Movies:")
    for i in distances[1:6]:  # Fix variable name from `distance` to `distances`
        print(movies.iloc[i[0]]['title'])  # Fetch movie title based on index


In [ ]:
recomand('Spider-Man')